In [ ]:
# dependencies import
from common_dependencies import *
logger = logging.getLogger(f'main.ae_model_train')

In [ ]:
# train params that can be easily changed
MAX_VAL = 1000
XSHIFT = 200
dataset_desc = {'train': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,0,MAX_VAL,60),SlidingCrop(1,1),XSHIFT),
                          DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,0,MAX_VAL,60),SlidingCrop(1,1),XSHIFT),
                         ),
                'val': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,60,MAX_VAL,20),SlidingCrop(1,1),XSHIFT),
                        DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,60,MAX_VAL,20),SlidingCrop(1,1),XSHIFT)
                       ),
                'test': (DatasetPartDescription(PATH_TO_DATA['run_1'],DataCrop(0,80,MAX_VAL,MAX_VAL),SlidingCrop(1,1),XSHIFT),
                         DatasetPartDescription(PATH_TO_DATA['run_2'],DataCrop(0,80,MAX_VAL,MAX_VAL),SlidingCrop(1,1),XSHIFT)
                        )}

In [ ]:
#входные и выходные данные
# reading
dataset = {'train':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['train'])])),
           'val':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['val'])])), 
           'test':dict(zip(['x','y','bin'], [np.array(list(gen)) for gen in chain_dataset_gens(dataset_desc['test'])]))}

# squueze datasets
for dataset_part_name, dataset_part in dataset.items():
    for data_part_name, data_part in dataset_part.items():
        if data_part_name == 'x':
            dataset[dataset_part_name][data_part_name] = data_part.reshape(-1,64)
        if data_part_name == 'y':
            dataset[dataset_part_name][data_part_name] = data_part.reshape(-1)

# displaying
logger.debug('Dataset')
for dataset_part_name, dataset_part in dataset.items():
    logger.debug('|'*8+dataset_part_name+'|'*8)
    for data_part_name, data_part in dataset_part.items():
        logger.debug(f'{data_part_name}.shape: {data_part.shape}')

In [ ]:
#выборка данных
# show parts took for learning
all_rects = {'run_1': {'train':None,'val':None,'test':None}, 
             'run_2': {'train':None,'val':None,'test':None}}
rects_colors = {'train':'red', 'val':'green', 'test':'yellow'}

for run_name in all_rects.keys():
    x_df, y_df = dw.get_x_and_y_data(*PATH_TO_DATA[run_name])
    x_df = None
    y_df = dw.roll_df(y_df, XSHIFT, 1)
    for dataset_part_name in all_rects[run_name].keys():
        # get all DatasetPartDescription for train, val or test
        dataset_part_desc = dataset_desc[dataset_part_name]
        # get all DatasetPartDescription for current run_name (run_1 or run_2)
        dataset_part_desc = [dataset_part for dataset_part in dataset_part_desc if re.findall(r'run_\d', dataset_part.data_path_tuple[0])[0] == run_name]
        # put rects list to all_rects[run_name][dataset_part_name]
        all_rects[run_name][dataset_part_name] = [Rectangle((dataset_part.file_data_crop.left, dataset_part.file_data_crop.top), 
                           dataset_part.file_data_crop.width, dataset_part.file_data_crop.height, 
                           facecolor=rects_colors[dataset_part_name], alpha=0.5) for dataset_part in dataset_part_desc]
    res_rects = list(itertools.chain(*[run_rects for run_rects_name, run_rects in all_rects[run_name].items()]))
    if res_rects:
        dw.draw_defects_map_with_rectangles_owerlap(y_df, res_rects, title = f'The parts took for learning from run_1 (red - train, green - validate, yellow - test)')

In [ ]:
#### всякие константы для последующей работы

#///////////////////////////////// для компиляции 

#CMP_learning_rate = 0.005 #0.0000002 # шаг сходимости back propogation
#CMP_solver = keras.optimizers.Adam(CMP_learning_rate) # оптимизатор
#CMP_solver = keras.optimizers.SGD(CMP_learning_rate) # оптимизатор
#CMP_loss_funcs = keras.losses.MeanSquaredError() #BinaryCrossentropy() 
#CMP_metrics = [keras.metrics.MeanSquaredError(name='MeanSquaredError')]
#///////////////////////////////// для колбэков

# для Early_stopping
ES_patience = 8 # кол-во эпох без улучшений
ES_min_delta = 0.000001 # минимальное улучшение параметра за cur_patience
ES_monitor_parametr =  'val_loss' # отслеживаемый параметр 
ES_save_best_weights = False # сохранять ли веса нейронки с лучшими результатами
#Es_mode = 'max'

   # для ReduceLROnPlateau
RLPOP_monitor_parametr = 'loss'  # отслеживаемый параметр 
RLPOP_factor = 0.2 # множитель для расчета нового шага сходимости (new_learning_rate = old_learning_rate*RLPOP_factor)
RLPOP_patience = 3 # кол-во эпох без улучшений
RLPOP_verbose = 0 # выводить ли прогресс изменения шага сходимости в его процессее
RLPOP_mode = 'auto' # выбирает, уменьшать шаг сходимости при росте величины или при её уменьшении
RLPOP_min_delta = 0.0001 # порог изменения отслеживаемого значения
RLPOP_cooldown = 1 # количество эпох до возобновления работы после изменения шага сходимости
RLPOP_min_lr = 0 # минимальное значение шага сходимости
    
#///////////////////////////////// для тренировки

#FIT_batch_size = 32 # размерpython concat lists bach при обучении/тестировании1
#FIT_shuffle = True # перемешивать ли данные
FIT_verbose = 0 # выводить ли прогресс обучения в его процессее
FIT_epochs = 60 # количество эпох обучения
#FIT_validation_split = 0.10 #0.20 # процент валидационных данных, отсекаемых из тестовой выборки

In [ ]:
'''#шараш-монтаж модели сети
HIDDEN_ACTIVATION = 'sigmoid' #'sigmoid'
OUTPUT_ACTIVATION = 'sigmoid'

enc_input = layers.Input((64,), name='enc_input')
d_1_1 = layers.Dense(64, activation=HIDDEN_ACTIVATION)(enc_input)
d_2_1 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(d_1_1)
d_2_2 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(d_1_1)
#d_2_3 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(d_1_1)
#d = layers.Dense(16, activation='linear')(d)
#d = layers.Dense(8, activation='linear')(d)
#d = layers.Dense(4, activation='linear')(d)
hidden_state_output = layers.Dense(2, activation=HIDDEN_ACTIVATION, name='hidden_state_output')(concatenate([d_2_1, d_2_2], axis=1))

d_3_1 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(hidden_state_output)
d_3_2 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(hidden_state_output)
#d_3_3 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(hidden_state_output)
#d = layers.Dense(8, activation='linear')(d)
#d = layers.Dense(16, activation='linear')(d)
#d = layers.Dense(32, activation='sigmoid')(d)
dec_output = layers.Dense(64, activation=OUTPUT_ACTIVATION, name='dec_output')(concatenate([d_3_1, d_3_2], axis=1))


model = keras.Model(enc_input, dec_output, name='ae')'''

'''model = keras.Sequential()
model.add(layers.Input((64,)))
model.add(layers.Dense(64, activation='linear', name='enc_input'))
model.add(layers.Dense(32, activation='linear'))
model.add(layers.Dense(16, activation='linear', name='hidden_state_output'))
model.add(layers.Dense(32, activation='linear'))
model.add(layers.Dense(64, activation='sigmoid', name='dec_output'))'''

'''model.compile(optimizer=CMP_solver, loss=CMP_loss_funcs)

print(model.summary())''';

In [ ]:
#визуализация сети в виде графа
'''tf.keras.utils.plot_model(
    model,
    show_shapes=True,
    show_dtype=False,
    show_layer_names=True,
    rankdir="TB",
    expand_nested=False,
    dpi=200,
    show_layer_activations=False,
    show_trainable=False,
)''';

In [ ]:

# Создание и настройка колбэков

callback_list = [] # массив колбэков до подачи в колбек "callbacklist"

callback_list.append(keras.callbacks.EarlyStopping(
            monitor = ES_monitor_parametr, 
            min_delta = ES_min_delta, 
            patience = ES_patience,
            restore_best_weights = ES_save_best_weights
            ))

callback_list.append(keras.callbacks.ReduceLROnPlateau(
            monitor = RLPOP_monitor_parametr, 
            factor = RLPOP_factor, 
            patience = RLPOP_patience, 
            verbose = RLPOP_verbose,
            mode = RLPOP_mode, 
            min_delta = RLPOP_min_delta, 
            cooldown = RLPOP_cooldown, 
            min_lr = RLPOP_min_lr
            ))

In [ ]:
'''PATH_TO_MODEL = 'networks/AE'
MODEL_VER = 4
MODEL_NUM = 1

for name in os.listdir(PATH_TO_MODEL):
    res = re.match(F'(id=v{MODEL_VER:02}n{MODEL_NUM:02}).*', name)
    if not res is None:
        PATH_TO_MODEL += '/' + res[0]
        break
        
print(PATH_TO_MODEL)

model = keras.models.load_model(PATH_TO_MODEL)

train=model.evaluate(dataset['train']['x'], dataset['train']['x'], batch_size = 32, verbose=0) 
val=model.evaluate(dataset['val']['x'], dataset['val']['x'], batch_size = 32, verbose=0)        
test=model.evaluate(dataset['test']['x'], dataset['test']['x'], batch_size = 32, verbose=0)

model_name = f"networks/AE/id=v{MODEL_VER:02}n{MODEL_NUM:02}_in(64)_hid(16)_loss_MSE=(train={train:.5f},val={val:.5f},test={test:.5f}).keras"

print(model_name)'''

In [ ]:
#id=v13n01_in(64)_hid(16)_loss_MSE=(train=0.00077,val=0.00054,test=0.00075)
min_train_loss = 0.00141
min_val_loss = 0.00098
num=1
for i in range(0,1000,10):
    tf.compat.v1.set_random_seed(i)
    tf.random.set_seed(i)
    np.random.seed(i)
    print(f'Seed: {i}',"|"*10)
    for learning_rate in [0.01,0.005]:
        print(f'Start learning rate: {learning_rate}',"|"*5)
        for batch_size in [32,]:
            print(f'\tBatch size: {batch_size}')
            #CMP_learning_rate = 0.005 #0.0000002 # шаг сходимости back propogation
            CMP_solver = keras.optimizers.Adam(learning_rate) # оптимизатор
            CMP_loss_funcs = keras.losses.MeanSquaredError() #BinaryCrossentropy() 
        
            HIDDEN_ACTIVATION = 'relu' #'sigmoid'
            OUTPUT_ACTIVATION = 'sigmoid'
            
            enc_input = layers.Input((64,), name='enc_input')
            d_1_1 = layers.Dense(64, activation=HIDDEN_ACTIVATION)(enc_input)
            
            d_2_1 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(d_1_1)
            d_2_2 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(d_1_1)
            #d_2_3 = layers.Dense(8, activation=HIDDEN_ACTIVATION)(d_1_1)

            #d_4_1 = layers.Dense(16, activation=HIDDEN_ACTIVATION)(concatenate([d_2_1, d_2_2], axis=1))
            #d_4_3 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(d_1_1)

            hidden_state_output = layers.Dense(12, activation=HIDDEN_ACTIVATION, name='hidden_state_output')(concatenate([d_2_1, d_2_2], axis=1))
        
            #d_5_1 = layers.Dense(16, activation=HIDDEN_ACTIVATION)(hidden_state_output)
            

            d_7_1 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(hidden_state_output)
            d_7_2 = layers.Dense(32, activation=HIDDEN_ACTIVATION)(hidden_state_output)
            #d_7_3 = layers.Dense(8, activation=HIDDEN_ACTIVATION)(d_6_2)


            dec_output = layers.Dense(64, activation=OUTPUT_ACTIVATION, name='dec_output')(concatenate([d_7_1,d_7_2], axis=1))
            
            model = keras.Model(enc_input, dec_output, name='ae')
            model.compile(optimizer=CMP_solver, loss=CMP_loss_funcs)

            history = model.fit(dataset['train']['x'], dataset['train']['x'],
                            batch_size = batch_size, 
                            epochs = FIT_epochs, 
                            verbose = FIT_verbose, 
                            shuffle=True,
                            validation_data = (dataset['val']['x'], dataset['val']['x']), 
                            callbacks = callback_list)
            res_loss = model.evaluate(dataset['test']['x'], dataset['test']['x'], batch_size = 32, verbose=0)
            
            cont = False

            model_name = f"networks/AE/id=v15n{num:02}_in(64)_hid(12)_loss_MSE=(train={history.history['loss'][-1]:.5f},val={history.history['val_loss'][-1]:.5f},test={res_loss:.5f}).keras"
            
            if history.history['loss'][-1] < min_train_loss and history.history['val_loss'][-1] < min_val_loss:
                min_train_loss = history.history['loss'][-1]
                min_val_loss = history.history['val_loss'][-1]
                cont = True
            if history.history['val_loss'][-1] < min_val_loss:
                min_val_loss = history.history['val_loss'][-1]
                cont = True
            if history.history['loss'][-1] < min_train_loss:
                min_train_loss = history.history['loss'][-1]
                cont = True
                
            print(f'\tEpochs: {len(history.history["loss"])}')
            print(f"\tloss_MSE=(train={history.history['loss'][-1]:.5f},val={history.history['val_loss'][-1]:.5f},test={res_loss:.5f})")
            
            
            if cont:
                model.save(model_name)
                num+=1
                cont=False
                #print(f'Start learning rate: {learning_rate}',"|"*5)
                #print(f'Batch size: {batch_size}')
                #print(f'Epochs: {len(history.history["loss"])}')
                #print(f"loss_MSE=(train={history.history['loss'][-1]:.5f},val={history.history['val_loss'][-1]:.5f},test={res_loss:.5f})")
                #print()
                continue
    print()

'''
Seed: 210 ||||||||||
Start learning rate: 0.005 |||||
Batch size: 16
Epochs: 24
loss_MSE=(train=0.00141,val=0.00098,test=0.00138)
'''

In [ ]:
#res_loss = model.evaluate(dataset['test']['x'], dataset['test']['x'], batch_size = 32)

In [ ]:
'''FONT_SIZE = 15
fig, ax = plt.subplots()

fig.set_figwidth(12)
fig.set_figheight(8)

ax.plot(history.history['loss'], 
         label='Train dataset',  linewidth=1.5, color='blue')
ax.plot(history.history[f'val_loss'], linestyle = '--', 
     label='Validation dataset',  linewidth=3, color='red')
ax.set_xlabel('Epoch number', fontsize=FONT_SIZE)
ax.set_ylabel(f'Loss value', fontsize=FONT_SIZE)
ax.set_title(f"Learning process loss plot. Test dataset value = {res_loss:.4f} (MSE)", fontsize=FONT_SIZE, pad=15)

ax.patch.set_alpha(0)

#  Устанавливаем форматирование делений:
ax.tick_params(axis='both', which='both', labelsize = FONT_SIZE)

# Вывод и настройка сетки
ax.minorticks_on()
ax.grid(which='major', linewidth=2)
ax.grid(which='minor', color = 'gray', linestyle = ':')

ax.legend(fontsize = FONT_SIZE, facecolor = "white", loc = 'upper right')

plt.show()''';

In [ ]:
#print(f"loss_MSE=(train={history.history['loss'][-1]:.5f},val={history.history['val_loss'][-1]:.5f},test={res_loss:.5f})")

In [ ]:
# сохранение модели в файл
#import h5py
#from tensorflow.keras.models import load_model
#model.save(f"networks/AE/id=v06n01_in(64)_hid(2)_loss_MSE=(train={history.history['loss'][-1]:.5f},val={history.history['val_loss'][-1]:.5f},test={res_loss:.5f}).keras")

In [ ]:
# loss_MSE=(train=0.01311,val=0.01137,test=0.01258) 2 parallel
# loss_MSE=(train=0.01461,val=0.01294,test=0.01408)
# 